In [3]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [4]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [5]:
def split_data(X, y):
    y_5_or_6 = (y == 5 )| (y == 6)
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2
    y_B = (y[y_5_or_6] == 6).astype(np.float32)
    return ((X[~y_5_or_6], y_A), (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B_full, y_train_B_full) = split_data(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_data(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_data(X_test, y_test)
X_train_B = X_train_B_full[:200]
y_train_B = y_train_B_full[:200]

In [6]:
tf.random.set_seed(42)
np.random.seed(42)

In [7]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape = [28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(units=n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [8]:
model_A.compile(loss = "sparse_categorical_crossentropy", metrics=["accuracy"], optimizer=keras.optimizers.SGD(lr = 1e-3))

In [9]:
history = model_A.fit(X_train_A, y_train_A, epochs=20, validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.5926 - accuracy: 0.8103 - val_loss: 0.3890 - val_accuracy: 0.8677
Epoch 2/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.3523 - accuracy: 0.8786 - val_loss: 0.3288 - val_accuracy: 0.8822
Epoch 3/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.3170 - accuracy: 0.8896 - val_loss: 0.3013 - val_accuracy: 0.8991
Epoch 4/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.2973 - accuracy: 0.8975 - val_loss: 0.2895 - val_accuracy: 0.9018
Epoch 5/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.2834 - accuracy: 0.9021 - val_loss: 0.2776 - val_accuracy: 0.9066
Epoch 6/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.2729 - accuracy: 0.9061 - val_loss: 0.2735 - val_accuracy: 0.9063
Epoch 7/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.2640 - accuracy: 0.9093 - val_loss: 0.2722 - val_accuracy:

In [10]:
model_A.save("my_model_A.h5")

In [11]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape = [28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(units=n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [12]:
model_B.compile(loss = "binary_crossentropy", metrics=["accuracy"], optimizer=keras.optimizers.SGD(lr = 1e-3))

In [13]:
history = model_B.fit(X_train_B, y_train_B, epochs=20, validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 0s 18ms/step - loss: 0.9573 - accuracy: 0.4650 - val_loss: 0.6314 - val_accuracy: 0.6004
Epoch 2/20
7/7 [==============================] - 0s 9ms/step - loss: 0.5692 - accuracy: 0.7450 - val_loss: 0.4784 - val_accuracy: 0.8529
Epoch 3/20
7/7 [==============================] - 0s 9ms/step - loss: 0.4503 - accuracy: 0.8650 - val_loss: 0.4102 - val_accuracy: 0.8945
Epoch 4/20
7/7 [==============================] - 0s 9ms/step - loss: 0.3879 - accuracy: 0.8950 - val_loss: 0.3647 - val_accuracy: 0.9178
Epoch 5/20
7/7 [==============================] - 0s 9ms/step - loss: 0.3435 - accuracy: 0.9250 - val_loss: 0.3300 - val_accuracy: 0.9320
Epoch 6/20
7/7 [==============================] - 0s 9ms/step - loss: 0.3081 - accuracy: 0.9300 - val_loss: 0.3019 - val_accuracy: 0.9402
Epoch 7/20
7/7 [==============================] - 0s 9ms/step - loss: 0.2800 - accuracy: 0.9350 - val_loss: 0.2804 - val_accuracy: 0.9422
Epoch 8/20
7/7 [=================

In [14]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation = "sigmoid"))

In [15]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [16]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

In [17]:
model_B_on_A.compile(loss = "binary_crossentropy", optimizer = "sgd", metrics =["accuracy"])

In [18]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs = 4, validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 18ms/step - loss: 0.4671 - accuracy: 0.7450 - val_loss: 0.3565 - val_accuracy: 0.8519
Epoch 2/4
7/7 [==============================] - 0s 9ms/step - loss: 0.2758 - accuracy: 0.9400 - val_loss: 0.2349 - val_accuracy: 0.9422
Epoch 3/4
7/7 [==============================] - 0s 9ms/step - loss: 0.1827 - accuracy: 0.9650 - val_loss: 0.1772 - val_accuracy: 0.9665
Epoch 4/4
7/7 [==============================] - 0s 9ms/step - loss: 0.1368 - accuracy: 0.9800 - val_loss: 0.1446 - val_accuracy: 0.9828


In [19]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True
    
optimizer = keras.optimizers.SGD(lr = 1e-4)
model_B_on_A.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs = 16, validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 [==============================] - 0s 18ms/step - loss: 0.1176 - accuracy: 0.9900 - val_loss: 0.1428 - val_accuracy: 0.9828
Epoch 2/16
7/7 [==============================] - 0s 9ms/step - loss: 0.1158 - accuracy: 0.9900 - val_loss: 0.1409 - val_accuracy: 0.9828
Epoch 3/16
7/7 [==============================] - 0s 9ms/step - loss: 0.1139 - accuracy: 0.9900 - val_loss: 0.1390 - val_accuracy: 0.9828
Epoch 4/16
7/7 [==============================] - 0s 9ms/step - loss: 0.1121 - accuracy: 0.9900 - val_loss: 0.1372 - val_accuracy: 0.9828
Epoch 5/16
7/7 [==============================] - 0s 9ms/step - loss: 0.1104 - accuracy: 0.9900 - val_loss: 0.1355 - val_accuracy: 0.9828
Epoch 6/16
7/7 [==============================] - 0s 9ms/step - loss: 0.1087 - accuracy: 0.9900 - val_loss: 0.1338 - val_accuracy: 0.9848
Epoch 7/16
7/7 [==============================] - 0s 9ms/step - loss: 0.1071 - accuracy: 0.9900 - val_loss: 0.1323 - val_accuracy: 0.9848
Epoch 8/16
7/7 [=================

In [20]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 1ms/step - loss: 0.1166 - accuracy: 0.9875


[0.11664953082799911, 0.987500011920929]